In [1]:
%cd ..

/home/jupyter-tim/ba-tim


In [ ]:
qaplib_results = {} # Empty results cache

In [392]:
from drlqap.taskgenerators import generators
import scipy.optimize
from drlqap.simplesolver import solve_qap_backtracking, solve_qap_maxgreedy, solve_qap_faq, solve_random, solve_partial_random, solve_best_of_k, mirror_solver
from drlqap.gurobi import solve_qap_gurobi
import numpy as np
from drlqap.qap import QAP
from drlqap.qaplib import load_qap
from drlqap.evaltools import load_checkpoints
from pathlib import Path
import math

In [193]:
importlib.reload(drlqap.simplesolver)

<module 'drlqap.simplesolver' from '/home/jupyter-tim/ba-tim/src/drlqap/simplesolver.py'>

In [17]:
mini_agents = load_checkpoints(Path('runs/dqn_dense_ms_ec_eps0_mini/lr5e-4_s2/'))

runs/dqn_dense_ms_ec_eps0_mini/lr5e-4_s2/checkpoint_0.pth
runs/dqn_dense_ms_ec_eps0_mini/lr5e-4_s2/checkpoint_1000.pth
runs/dqn_dense_ms_ec_eps0_mini/lr5e-4_s2/checkpoint_2000.pth
runs/dqn_dense_ms_ec_eps0_mini/lr5e-4_s2/checkpoint_3000.pth
runs/dqn_dense_ms_ec_eps0_mini/lr5e-4_s2/checkpoint_4000.pth
runs/dqn_dense_ms_ec_eps0_mini/lr5e-4_s2/checkpoint_end.pth


In [18]:
a2c_agents = load_checkpoints(Path('runs/a2c_ms100x_mediumrandoms/lr4e-5_s3'))

runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_0.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_1000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_2000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_3000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_4000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_5000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_6000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_7000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_8000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_9000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_10000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_11000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_12000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_13000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_14000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_s3/checkpoint_15000.pth
runs/a2c_ms100x_mediumrandoms/lr4e-5_

In [19]:
a2c_longer_agents = load_checkpoints(Path('runs/a2c_ms100x_mediumrandoms/lr2e-5_s1'))

runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_0.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_1000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_2000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_3000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_4000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_5000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_6000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_7000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_8000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_9000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_10000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_11000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_12000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_13000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_14000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_15000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_

In [20]:
dqn_small = load_checkpoints(Path('runs/a2c_ms100x_mediumrandoms/lr2e-5_s1'))

runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_0.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_1000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_2000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_3000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_4000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_5000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_6000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_7000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_8000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_9000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_10000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_11000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_12000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_13000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_14000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_s1/checkpoint_15000.pth
runs/a2c_ms100x_mediumrandoms/lr2e-5_

In [21]:
def evaluate_set(solver, problem):
    values = []
    test_set = generators[problem].test_set()
    for qap in test_set:
        v_solver, assignment = solver(qap)
        v = qap.compute_value(assignment)
        if not np.isclose(v, v_solver):
            print (f"solver outputs incorrect value (got {v_solver}, actual {v})")
        values.append(v)
    return np.mean(values)

In [115]:
def evaluate_comparison(solvers, problem, cached_results=None):
    results = cached_results or {}
    for name, solver in solvers.items():
        if name not in results:
            print(f"Evaluating {name}...")
            values = {}
        else:
            values = results[name]
        test_set = generators[problem].test_set()
        for qap in test_set:
            if qap.name not in values:
                print(f"Evaluating {name} on {qap.name}...")
                v_solver, assignment = solver(qap)
                if not v_solver or not assignment:
                    print(f"{name} could not solve {qap.name}")
                else:
                    v = qap.compute_value(assignment)
                    if not np.isclose(v, v_solver):
                        print (f"solver {name} outputs incorrect value on {qap.name} (got {v_solver}, actual {v})")
                    values[qap.name] = float(v)
                
        results[name] = values
    return results
    

In [470]:
solvers = {
    'faq': solve_qap_faq,
    'a2c': a2c_agents[-1].solve,
    'dqn_mini': mini_agents[-1].solve,
    'random': solve_random,
    'a2c_longer': a2c_longer_agents[-1].solve,
    'max_greedy': solve_qap_maxgreedy,
    'dqn_sm': load_checkpoints(Path('runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1'))[-1].solve,
    'dqn_md': load_checkpoints(Path('runs/dqn_dense_ms_ec_eps0_mediumrandoms/lr5e-4_s1'))[-1].solve,
    'rf': load_checkpoints(Path('runs/reinforce_ms100x_smallrandoms/lr1e-4_s4/'))[-1].solve,
    'last7opt': lambda qap: solve_partial_random(qap, 7),
    'rand1k': lambda qap: solve_best_of_k(qap, 1000),
    'rand5k': lambda qap: solve_best_of_k(qap, 5000),
    'rand10k': lambda qap: solve_best_of_k(qap, 10000),
    'a2c_sm': load_checkpoints(Path('runs/a2c_ms100x_smallrandoms/lr4e-5_s2'))[-1].solve}

solvers['rf_m'] = mirror_solver(solvers['rf'])
solvers['a2c_m'] = mirror_solver(solvers['a2c'])

runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_0.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_1000.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_2000.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_3000.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_4000.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_5000.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_6000.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_7000.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_8000.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_9000.pth
runs/dqn_dense_ms_ec_eps0_smallrandoms/lr5e-4_s1/checkpoint_end.pth
runs/dqn_dense_ms_ec_eps0_mediumrandoms/lr5e-4_s1/checkpoint_0.pth
runs/dqn_dense_ms_ec_eps0_mediumrandoms/lr5e-4_s1/checkpoint_1000.pth
runs/dqn_dense_ms_ec_eps0_mediumrandoms/lr5e-4_s1/checkpoint_2000.pth
runs/dqn_dense_ms_ec_eps0_mediumrandom

In [471]:
qaplib_results = evaluate_comparison(solvers, 'qaplib_all_150_normalized', cached_results=qaplib_results)

Evaluating a2c_sm...
Evaluating a2c_sm on bur26a...
Evaluating a2c_sm on bur26b...
Evaluating a2c_sm on bur26c...
Evaluating a2c_sm on bur26d...
Evaluating a2c_sm on bur26e...
Evaluating a2c_sm on bur26f...
Evaluating a2c_sm on bur26g...
Evaluating a2c_sm on bur26h...
Evaluating a2c_sm on chr12a...
Evaluating a2c_sm on chr12b...
Evaluating a2c_sm on chr12c...
Evaluating a2c_sm on chr15a...
Evaluating a2c_sm on chr15b...
Evaluating a2c_sm on chr15c...
Evaluating a2c_sm on chr18a...
Evaluating a2c_sm on chr18b...
Evaluating a2c_sm on chr20a...
Evaluating a2c_sm on chr20b...
Evaluating a2c_sm on chr20c...
Evaluating a2c_sm on chr22a...
Evaluating a2c_sm on chr22b...
Evaluating a2c_sm on chr25a...
Evaluating a2c_sm on els19...
Evaluating a2c_sm on esc128...
Evaluating a2c_sm on esc16a...
Evaluating a2c_sm on esc16b...
Evaluating a2c_sm on esc16c...
Evaluating a2c_sm on esc16d...
Evaluating a2c_sm on esc16e...
Evaluating a2c_sm on esc16g...
Evaluating a2c_sm on esc16h...
Evaluating a2c_sm o

In [450]:
ngm_g5k_results = {
"bur26a": 5621774, 
"bur26b": 3927943, 
"bur26c": 5608065, 
"bur26d": 3962317, 
"bur26e": 5536142, 
"bur26f": 3949711, 
"bur26g": 10433439, 
"bur26h": 7348866, 
"chr12a": 14940, 
"chr12b": 14984, 
"chr12c": 16346, 
"chr15a": 20442, 
"chr15b": 22048, 
"chr15c": 24190, 
"chr18a": 33124, 
"chr18b": 2504, 
"chr20a": 5178, 
"chr20b": 5766, 
"chr20c": 49770, 
"chr22a": 9348, 
"chr22b": 9006, 
"chr25a": 11648, 
"els19": 27029748, 
"esc16a": 78, 
"esc16b": 292, 
"esc16c": 174, 
"esc16d": 20, 
"esc16e": 32, 
"esc16f": 0, 
"esc16g": 32, 
"esc16h": 1004, 
"esc16i": 18, 
"esc16j": 8, 
"esc32a": 298, 
"esc32b": 368, 
"esc32c": 754, 
"esc32d": 284, 
"esc32e": 2, 
"esc32g": 10, 
"esc32h": 534, 
"esc64a": 200, 
"esc128": 242, 
"had12": 1700, 
"had14": 2866, 
"had16": 3902, 
"had18": 5558, 
"had20": 7300, 
"kra30a": 114410, 
"kra30b": 118130, 
"kra32": 120930, 
"lipa20a": 3853, 
"lipa20b": 33125, 
"lipa30a": 13631, 
"lipa30b": 187607, 
"lipa40a": 32454, 
"lipa40b": 601848, 
"lipa50a": 63671, 
"lipa50b": 1523856, 
"lipa60a": 109595, 
"lipa60b": 3208501, 
"lipa70a": 173220, 
"lipa70b": 5890161, 
"lipa80a": 257663, 
"lipa80b": 9983040, 
"lipa90a": 366508, 
"lipa90b": 16076956, 
"nug12": 634, 
"nug14": 1156, 
"nug15": 1318, 
"nug16a": 1836, 
"nug16b": 1396, 
"nug17": 1980, 
"nug18": 2242, 
"nug20": 2936, 
"nug21": 2916, 
"nug22": 4298, 
"nug24": 4234, 
"nug25": 4420, 
"nug27": 6208, 
"nug28": 6128, 
"nug30": 7294, 
"rou12": 264898, 
"rou15": 403872, 
"rou20": 817776, 
"scr12": 36292, 
"scr15": 68768, 
"scr20": 154636, 
"sko42": 18716, 
"sko49": 27554, 
"sko56": 40684, 
"sko64": 56222, 
"sko72": 76870, 
"sko81": 104710, 
"sko90": 132942, 
"sko100a": 172810, 
"sko100b": 175588, 
"sko100c": 169806, 
"sko100d": 170816, 
"sko100e": 170958, 
"sko100f": 169986, 
"ste36a": 16768, 
"ste36b": 43248, 
"ste36c": 12988352, 
"tai12a": 255158, 
"tai12b": 47252044, 
"tai15a": 436968, 
"tai15b": 52871608, 
"tai17a": 544754, 
"tai20a": 806382, 
"tai20b": 140704160, 
"tai25a": 1352912, 
"tai25b": 518647040, 
"tai30a": 2065706, 
"tai30b": 896379008, 
"tai35a": 2786748, 
"tai35b": 377687744, 
"tai40a": 3610604, 
"tai40b": 917498816, 
"tai50a": 5677282, 
"tai50b": 614638528, 
"tai60a": 8281996, 
"tai60b": 862969152, 
"tai64c": 2133738, 
"tai80a": 15283138, 
"tai80b": 1120577408, 
"tai100a": 23644528, 
"tai100b": 1612020992, 
"tai150b": 628349568, 
"tho30": 185622, 
"tho40": 304878, 
"tho150": 9557766, 
"wil50": 53418, 
"wil100": 294172,
}

qaplib_results["ngm_g5k"] = ngm_g5k_results

In [451]:
ngm_results = {
"bur26a": 5684628,
"bur26b": 4063246,
"bur26c": 5638641,
"bur26d": 3994147,
"bur26e": 5666202,
"bur26f": 3954977,
"bur26g": 10855165,
"bur26h": 7670546,
"chr12a": 27556,
"chr12b": 29396,
"chr12c": 34344,
"chr15a": 50272,
"chr15b": 52082,
"chr15c": 38568,
"chr18a": 83026,
"chr18b": 4810,
"chr20a": 10728,
"chr20b": 9962,
"chr20c": 115128,
"chr22a": 16410,
"chr22b": 15876,
"chr25a": 18950,
"els19": 34880280,
"esc16a": 88,
"esc16b": 308,
"esc16c": 184,
"esc16d": 40,
"esc16e": 48,
"esc16f": 0,
"esc16g": 50,
"esc16h": 1036,
"esc16i": 26,
"esc16j": 16,
"esc32a": 428,
"esc32b": 424,
"esc32c": 844,
"esc32d": 288,
"esc32e": 42,
"esc32g": 28,
"esc32h": 592,
"esc64a": 250,
"esc128": 238,
"had12": 1790,
"had14": 2922,
"had16": 4150,
"had18": 5780,
"had20": 7334,
"kra30a": 114410,
"kra30b": 118130,
"kra32": 121340,
"lipa20a": 3929,
"lipa20b": 33907,
"lipa30a": 13841,
"lipa30b": 192356,
"lipa40a": 32666,
"lipa40b": 616656,
"lipa50a": 64100,
"lipa50b": 1543264,
"lipa60a": 110094,
"lipa60b": 3269504,
"lipa70a": 173862,
"lipa70b": 5978316,
"lipa80a": 258402,
"lipa80b": 10173155,
"lipa90a": 367193,
"lipa90b": 16194745,
"nug12": 720,
"nug14": 1210,
"nug15": 1482,
"nug16a": 1836,
"nug16b": 1580,
"nug17": 2004,
"nug18": 2312,
"nug20": 2936,
"nug21": 2916,
"nug22": 4616,
"nug24": 4234,
"nug25": 4420,
"nug27": 6332,
"nug28": 6128,
"nug30": 7608,
"rou12": 321082,
"rou15": 469592,
"rou20": 897348,
"scr12": 44400,
"scr15": 81344,
"scr20": 182882,
"sko42": 20192,
"sko49": 28712,
"sko56": 42182,
"sko64": 60368,
"sko72": 79716,
"sko81": 107588,
"sko90": 137402,
"sko100a": 180972,
"sko100b": 180774,
"sko100c": 175740,
"sko100d": 175096,
"sko100e": 176010,
"sko100f": 173552,
"ste36a": 16648,
"ste36b": 43248,
"ste36c": 12988352,
"tai12a": 259014,
"tai12b": 65138752,
"tai15a": 467812,
"tai15b": 495479040,
"tai17a": 630644,
"tai20a": 896518,
"tai20b": 237607744,
"tai25a": 1393248,
"tai25b": 730775168,
"tai30a": 2065706,
"tai30b": 1359600384,
"tai35a": 2886132,
"tai35b": 455718176,
"tai40a": 3610604,
"tai40b": 1053339520,
"tai50a": 5891066,
"tai50b": 764856128,
"tai60a": 8596094,
"tai60b": 994559424,
"tai64c": 5703540,
"tai80a": 15648708,
"tai80b": 1275809408,
"tai100a": 24077728,
"tai100b": 1853681152,
"tai150b": 653429440,
"tho30": 187062,
"tho40": 313026,
"tho150": 9702946,
"wil50": 55390,
"wil100": 295418,
}

qaplib_results["ngm"] = ngm_results

In [452]:
# Qaplib solutions
qaplib_results["known"] = {}
for qap in generators["qaplib_all_150_normalized"].test_set():
    if qap.known_solution:
        qaplib_results["known"][qap.name] = qap.known_solution.value

In [221]:
from collections import defaultdict

def results_by_qap(results_by_solver):
    out = defaultdict(dict)
    for solver, results in results_by_solver.items():
        for qap, value in results.items():
            out[qap][solver] = value
    return dict(out)

In [479]:
benchmark = 'known'
compare = ['ngm_g5k', 'a2c_sm']
count_by_solver = defaultdict(lambda: 0)
for qap, results in results_by_qap(qaplib_results).items():
    # Skip qaps where one of the solvers has no result
    incomplete_results = False
    for solver in compare + [benchmark]:
        if solver not in results:
            print(solver, "has no data on", qap)
            incomplete_results = True
    if incomplete_results:
        print("Skipping", qap)
        continue

    ordered_results = sorted([s for s in results if s in compare], key=lambda s: results[s])
    base = results[benchmark]
    gap_to_benchmark = [f"{solver:>8}: {(results[solver]/base-1)*100:>5.1f}" for solver in ordered_results]
    print(f'{f"[{qap}]":>11} {base:>10} | {", ".join(gap_to_benchmark)}')
    count_by_solver[ordered_results[0]] += 1

print("\nNumber of times best value achieved")
for solver, count in count_by_solver.items():
    print(solver, count)

   [bur26a]    5426670 |  ngm_g5k:   3.6,   a2c_sm:   4.3
   [bur26b]    3817852 |  ngm_g5k:   2.9,   a2c_sm:   5.3
   [bur26c]    5426795 |   a2c_sm:   3.3,  ngm_g5k:   3.3
   [bur26d]    3821225 |  ngm_g5k:   3.7,   a2c_sm:   6.9
   [bur26e]    5386879 |  ngm_g5k:   2.8,   a2c_sm:   5.2
   [bur26f]    3782044 |  ngm_g5k:   4.4,   a2c_sm:   6.7
   [bur26g]   10117172 |  ngm_g5k:   3.1,   a2c_sm:   4.7
   [bur26h]    7098658 |  ngm_g5k:   3.5,   a2c_sm:   5.3
   [chr12a]       9552 |  ngm_g5k:  56.4,   a2c_sm:  57.6
   [chr12b]       9742 |  ngm_g5k:  53.8,   a2c_sm:  76.0
   [chr12c]      11156 |   a2c_sm:  15.0,  ngm_g5k:  46.5
   [chr15a]       9896 |  ngm_g5k: 106.6,   a2c_sm: 138.4
   [chr15b]       7990 |   a2c_sm:  68.0,  ngm_g5k: 175.9
   [chr15c]       9504 |   a2c_sm:  64.7,  ngm_g5k: 154.5
   [chr18a]      11098 |   a2c_sm:  90.1,  ngm_g5k: 198.5
   [chr18b]       1534 |   a2c_sm:  11.9,  ngm_g5k:  63.2
   [chr20a]       2192 |   a2c_sm:  88.6,  ngm_g5k: 136.2
   [chr20b]   

# Gaps by class

In [457]:
for qap in generators["qaplib_all_150_normalized"].test_set():
    if not qap.known_solution: print(qap.name)

tai10a
tai10b


In [458]:
rgm_results_by_class = {
    'bur': 0.071,
    'chr': 1.124,
    'esc': 0.328,
    'had': 0.062,
    'kra': 0.150,
    'lipa': 0.133,
    'nug': 0.097,
    'rou': 0.134,
    'scr': 0.455,
    'sko': 0.106,
    'ste': 1.341,
    'tai': 0.173,
    'tho': 0.207,
    'wil': 0.081,
    'els': math.inf
}

In [459]:
import re
def get_qaplib_class(name):
    return re.match("[a-z]*", name).group()

qap_classes = defaultdict(list)
for qap in generators["qaplib_all_150_normalized"].test_set():
    qap_classes[get_qaplib_class(qap.name)].append(qap)

In [466]:
def gaps_by_class(solvers, reference="known", qap_size=64):
    meta_gaps = defaultdict(list)
    for category, qaps in qap_classes.items():
        if category == 'els':
            continue
        # Skip qaps where one of the solvers has no result
        results = {}
        results['rgm'] = rgm_results_by_class[category]
        meta_gaps['rgm'].append(rgm_results_by_class[category])
        for solver in solvers:
            if solver in results:
                continue
            gaps = []
            for qap in qaps:
                if qap.size < qap_size:
                    if qap.name in qaplib_results[reference]:
                        base_val = qaplib_results[reference][qap.name]
                        gaps.append(qaplib_results[solver][qap.name] / base_val - 1)
            mean_gap = np.mean(gaps)
            results[solver] = mean_gap
            meta_gaps[solver].append(mean_gap)
        
        ordered = sorted(solvers, key=lambda s: results[s])
        gaps_strings = [f"{solver:>8}: {results[solver]*100:>5.1f}" for solver in ordered]
        min_size = min([q.size for q in qaps])
        max_size = max([q.size for q in qaps])
        print(f'{f"[{category}, {min_size}-{max_size}]":<13}{",".join(gaps_strings)}')
        
    for solver in solvers:
        print(f"{solver} mean: {np.mean(meta_gaps[solver])*100:.2f}")

In [481]:
gaps_by_class(["rf_m", "rf", "a2c", 'a2c_m', 'ngm_g5k', 'a2c_sm', 'faq'], qap_size=64)

[bur, 26-26]      faq:   0.2, ngm_g5k:   3.4,      rf:   4.0,  a2c_sm:   5.2,   a2c_m:   5.5,    rf_m:  14.3,     a2c:  14.4
[chr, 12-25]      faq:  54.9,  a2c_sm:  74.9,    rf_m:  98.7,     a2c: 104.1, ngm_g5k: 121.3,   a2c_m: 212.1,      rf: 263.8
[esc, 16-128]  a2c_sm:  16.9,     a2c:  18.5,    rf_m:  19.2, ngm_g5k:  32.0,     faq:  32.0,   a2c_m: 178.0,      rf: 255.3
[had, 12-20]      faq:   0.8,      rf:   3.9, ngm_g5k:   4.4,  a2c_sm:   4.8,   a2c_m:   5.2,     a2c:   7.0,    rf_m:   9.7
[kra, 30-32]      faq:   5.7,      rf:  28.5,  a2c_sm:  29.1,    rf_m:  29.4,   a2c_m:  30.3, ngm_g5k:  31.4,     a2c:  36.6
[lipa, 20-90]     faq:   2.5,  a2c_sm:  12.2,      rf:  13.6, ngm_g5k:  14.1,   a2c_m:  14.3,     a2c:  14.3,    rf_m:  14.4
[nug, 12-30]      faq:   3.0,      rf:  15.2,  a2c_sm:  16.1, ngm_g5k:  16.3,   a2c_m:  16.6,    rf_m:  22.4,     a2c:  23.3
[rou, 12-20]      faq:   3.8, ngm_g5k:  13.1,    rf_m:  14.3,   a2c_m:  14.7,     a2c:  15.1,  a2c_sm:  17.0,      rf:  20.2
